In [1]:
import numpy as np
import matplotlib.pyplot as plt
from skimage.morphology import binary_dilation, binary_erosion, disk
from skimage.filters import gaussian
import cv2

In [2]:
import os

def process_images_in_folder(folder_path, output_folder):
    # Verificar si la ruta de salida existe, si no, crearla
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Obtener la lista de archivos en el directorio
    files = os.listdir(folder_path)
    
    for file in files:
        if file.endswith('.jpg') or file.endswith('.png') or file.endswith('.jpeg'):
            # Cargar la imagen
            image_path = os.path.join(folder_path, file)
            mask = cv2.imread(image_path, 0)

            if mask is not None:
                # Aplicar el algoritmo Canny para detección de bordes
                edges = cv2.Canny(mask, 10, 100)  # Los valores 30 y 150 son umbrales para el algoritmo Canny

                # Now, create a set of heatmaps that contains only a 1 at the coordinate
                img_h, img_w = edges.shape
                coords = [(111,111), (111,0), (0,55), (0,0), (0,111), (55,0), (55,111)]
                num_coords = len(coords)
                heatmaps = np.zeros((num_coords, img_h, img_w))

                for i in range(num_coords):
                    current_coord = coords[i]
                    heatmaps[i, int(current_coord[0]), int(current_coord[1])] = 1

                # Now, apply a gaussian filter to each heatmap
                var = 20
                gaussian_heatmaps = gaussian(heatmaps, [0, var, var])

                # Now, use the contour as a mask of the heatmaps
                masked_heatmaps = np.zeros_like(gaussian_heatmaps)
                for i in range(num_coords):
                    masked_heatmaps[i] = edges * gaussian_heatmaps[i]
                    
                    # Renormalize so the values are between 0 and 1
                    max_val = np.max(masked_heatmaps[i])
                    masked_heatmaps[i] = masked_heatmaps[i] / max_val

                # Generate a folder with the name of the image that will contain the landmarks
                new_folder = os.path.join(output_folder, (file.replace('mask', 'landmark')).replace('.jpg', ''))
                if not os.path.exists(new_folder):
                    os.makedirs(new_folder)

                # Generate an image with each landmark as a 1 channel image
                for i in range(len(masked_heatmaps)):
                    heatmap = masked_heatmaps[i]
                    heatmap = (heatmap * 255).astype(np.uint8)
                    heatmap = cv2.cvtColor(heatmap, cv2.COLOR_GRAY2RGB)
                    cv2.imwrite(os.path.join(new_folder, '{}.jpg'.format(i)), heatmap)

        else:
            print(f"No se pudo cargar la imagen {file} en la ruta especificada.")

# Llama a la función para procesar las imágenes en el directorio de entrada
for i in ['TRAIN', 'TEST', 'VAL']:
    input_folder_path = "../Mascaras/Mask/{}".format(i)
    output_folder_path = "./Landmarks/{}".format(i)
    process_images_in_folder(input_folder_path, output_folder_path)

In [2]:
import os

# Contamos la cantidad de imágenes en la carpeta Frames y en la carpeta Landmarks
print("Cantidad de imágenes en la carpeta Frames:")
print('Train: ', len(os.listdir('../Frames/TRAIN')))
print('Test: ', len(os.listdir('../Frames/TEST')))
print('Val: ', len(os.listdir('../Frames/VAL')))
print()
print("Cantidad de imágenes en la carpeta Landmarks:")
print('Train: ', len(os.listdir('./Landmarks/TRAIN')))
print('Test: ', len(os.listdir('./Landmarks/TEST')))
print('Val: ', len(os.listdir('./Landmarks/VAL')))

Cantidad de imágenes en la carpeta Frames:
Train:  14920
Test:  2552
Val:  2574

Cantidad de imágenes en la carpeta Landmarks:
Train:  14920
Test:  2552
Val:  2574
